In [1]:
type ∨[P, Q] = Either[P, Q]
type ∧[P, Q] = Tuple2[P, Q] 
type ⟶[P, Q] = P => Q
type ⊥ = Nothing
type True = Unit
type ¬[P] = P => ⊥

defined type ∨
defined type ∧
defined type ⟶
defined type ⊥
defined type True
defined type ¬

In [2]:
type Or[P, Q] = Either[P, Q]
type And[P, Q] = Tuple2[P, Q]
type Implies[P, Q] = P => Q
type False = Nothing
type True = Unit
type Not[P] = P => ⊥

defined type Or
defined type And
defined type Implies
defined type False
defined type True
defined type Not

In [3]:
class Inhabitant{ x => 
    // Knight(x) -- `x` is a Knight
    // 
    type Knight
    
    // Knave(x) -- `x` is a Knave (i.e. is not a Knight)
    // 
    type Knave = ¬[Knight] // Knight => Nothing
    
    // Says(x, P) -- `x` says that `P` holds, i.e. asserts proposition `P`
    // 
    type Says[P]
}

defined class Inhabitant

In [4]:
trait KnightsKnaves{
    // P1. Inhabitants are knights or knaves
    // 
    //     ∀ x. Inhabitant(x) ⟶ Knight(x) ∨ Knave(x) 
    // 
    def P1(x: Inhabitant): x.Knight ∨ x.Knave
    
    // In Scala 3
    // val P1: (x: Inhabitant) => Either[x.Knight, x.Knave]
    
    // P2. Knights are truth tellers
    // 
    //     ∀ P. ∀ x. Knight(x) ⟶ Says(x, P) ⟶ P
    // 
    def P2[P](x: Inhabitant): x.Knight => x.Says[P] => P
    
    // In Scala 3
    // val P2: [P] => (x: Inhabitant) => x.Knight => x.Says[P] => P
    
    // P3. Knaves are persistent liers
    // 
    //     ∀ P. ∀ x. Knight(x) ⟶ Says(x, P) ⟶ ¬P
    // 
    def P3[P](x: Inhabitant): x.Knave => x.Says[P] => ¬[P]
}

defined trait KnightsKnaves

# Puzzles Introducción

Primero se consideran 5 preguntas que servirán como introduccion a la logica 
knight-knave para aquellos que no estén familiarizados con ella y como un
breve curso recordatorio para los que lo están.

#### 1. Is it possible for any inhabitant of this island to claim that he is a knave?

No; no inhabitant can claim to be a knave because no knight would lie and say he is a knave and no knave would truthfully admit to being a knave.

In [10]:
/* EJEMPLO RESUELTO - PUZZLE1 EJEMPLO*/

In [11]:
def puzzle1(premises: KnightsKnaves)(x: Inhabitant): x.Says[x.Knave] => Nothing = ???

defined function puzzle1

In [12]:
def puzzle1(premises: KnightsKnaves)(x: Inhabitant): Not[x.Says[x.Knave]] = ???

defined function puzzle1

In [21]:
//Ejemplo entero hecho por mi y entendido
def puzzle1(premises: KnightsKnaves)(x: Inhabitant): x.Says[x.Knave] => Nothing = 
    ({xsay: x.Says[x.Knave] =>
        (premises.P1(x) match{
            case Left(xKnight: x.Knight) => 
                //Siguiendo el ejemplo del profesor
                //def P2[P](x: Inhabitant): x.Knight => x.Says[P] => P
                //x_L1 es una funcion que dado un:
                //x.Knight devuelve una funcion que dado un:
                //x.Says[x.Knave] devuelve un x.Knave
                //O lo que es lo mismo, x_L1 es una funcion de tipo premises.P2[x.Knave](x)
                val x_L1: x.Knight => x.Says[x.Knave] => x.Knave = premises.P2[x.Knave](x)
                val x_L2: x.Says[x.Knave] => x.Knave = x_L1(xKnight)
                val x_L3: x.Knight => Nothing = x_L2(xsay)
                x_L3(xKnight): Nothing
            case Right(xKnave: x.Knave) => 
                //Siguiendo el ejemplo del profesor
                //def P3[P](x: Inhabitant): x.Knave => x.Says[P] => ¬[P]
                val x_R1: x.Knave => x.Says[x.Knave] => Not[x.Knave] = premises.P3[x.Knave](x)
                val x_R2: x.Says[x.Knave] => Not[x.Knave] = x_R1(xKnave)
                //Transformo x.Says[x.Knave] => Not[x.Knave] en x.Says[x.Knave] => x.Knave => Nothing
                val x_R3: x.Knave => Nothing = x_R2(xsay)
                x_R3(xKnave): Nothing
                
        }) : Nothing// Y aqui ya puedo quitar los ??? porque ya he obtenido una solucion de tipo Nothing dentro del {}
    }): (x.Says[x.Knave] => Nothing)

defined function puzzle1

In [22]:
//Ejemplo entero hecho por mi y entendido - SIMPLIFICADO
def puzzle1(premises: KnightsKnaves)(x: Inhabitant): x.Says[x.Knave] => Nothing = 
    ({xsay =>
        (premises.P1(x) match{
            case Left(xKnight) => 
                val x_L1: x.Knight => x.Says[x.Knave] => x.Knave = premises.P2[x.Knave](x)
                val x_L2: x.Says[x.Knave] => x.Knave = x_L1(xKnight)
                val x_L3: x.Knight => Nothing = x_L2(xsay)
                x_L3(xKnight)
            case Right(xKnave) => 
                val x_R1: x.Knave => x.Says[x.Knave] => Not[x.Knave] = premises.P3[x.Knave](x)
                val x_R2: x.Says[x.Knave] => Not[x.Knave] = x_R1(xKnave)
                val x_R3: x.Knave => Nothing = x_R2(xsay)
                x_R3(xKnave)
                
        }) 
    })

defined function puzzle1

In [7]:
implicit class SomeSugar(P: KnightsKnaves){    
    def eitherKnightOrKnave(x: Inhabitant): x.Knight Or x.Knave = P.P1(x)
    def knightsAreTruthTellers[P](x: Inhabitant) = P.P2[P](x)
    def knavesAreLiers[P](x: Inhabitant) = P.P3[P](x)
    
    def noKnightLies[P](x: Inhabitant): x.Knight => x.Says[Not[P]] => Not[P] = 
        xIsKnight => xSaysNotP => p => 
            P.P2[Not[P]](x)(xIsKnight)(xSaysNotP)(p)
    
    def noKnaveTellsTruth[P](x: Inhabitant): x.Knave => x.Says[P] => P => False = 
        xIsKnave => xSaysP => p => 
            P.P3[P](x)(xIsKnave)(xSaysP)(p)
}

defined class SomeSugar

In [8]:
// { P1, P2, P3 } ⊢ ∀ x. Inhabitant(x) ⟶ ¬ Says(x, Knave(x))

def puzzle1(premises: KnightsKnaves)(x: Inhabitant): Not[x.Says[x.Knave]] =
    xSaysIsKnave =>
        premises.eitherKnightOrKnave(x).fold(
            xIsKnight => 
                // no knight would lie
                premises.noKnightLies(x)(xIsKnight)(xSaysIsKnave)(xIsKnight),
            xIsKnave => 
                // no knave would tell the truth
                premises.noKnaveTellsTruth(x)(xIsKnave)(xSaysIsKnave)(xIsKnave)
        )

defined function puzzle1

In [9]:
implicit class F1Ext[P](np: P => Nothing){
    def contradicts(p: P): Nothing = np(p)
}

// Funcion contradiccion: dado un predicado, devuelvo Nothing
// Si tengo x.Knight => Nothing entonces eso es una contradiccion de x es Knight

defined class F1Ext

In [10]:
def puzzle1(premises: KnightsKnaves)(x: Inhabitant): Not[x.Says[x.Knave]] =
    xSaysIsKnave =>
        premises.eitherKnightOrKnave(x).fold(
            { xIsKnight => 
                // no knight would lie
                val xIsNotKnight = premises.noKnightLies(x)(xIsKnight)(xSaysIsKnave: x.Says[x.Knave]): (x.Knight => Nothing)
                // xIsNotKnight devuelve una funcion que dado un x.Knight devuelvo un Nothing
                // Si a xIsNotKnight le paso un x.Knight, entonces devuelvo Nothing
                xIsNotKnight.contradicts(xIsKnight) // Esto es lo mismo que: xIsNotKinght(xIsKnight)
            },
            xIsKnave => 
                // no knave would tell the truth
                (premises.noKnaveTellsTruth(x)(xIsKnave)(xSaysIsKnave): Not[x.Knave]).contradicts(xIsKnave)
                // Si tengo que x es un Knave y dice que es un Knave (Knave mienten) entonces es que x NO es un Knave
                // Osea que de (xIsKnave)(xSaysIsKnave) obtengo una funcion que dado un x.Knave devuelvo un Nothing
                // Si x es Knave y dice que es un Knave (como esta mintiendo), se contradice y x NO es un Knave 
        )

defined function puzzle1